In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [79]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype = float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    m = vec.shape[0]
    vec = vec.reshape((m, 1)) #had to expand dims to use np.transpose
    if m == 1:
        e = np.asarray([1])
        e = e.reshape((e.shape[0], 1))
        return np.asarray(vec), e
    xlen = length(vec)
    y = np.zeros(vec.shape, dtype = float)
    y[0] = xlen
    
    numerator = 0 #avoiding the roundoff error
    for xi in vec[1:]:
        numerator -= (xi**2)
    denominator = vec[0] + xlen
    diff_x1_y = numerator/denominator
    diff_rest = vec[1:]
    diff = np.insert(diff_rest, 0, diff_x1_y)
    difflen = length(diff)
    
    u = np.divide(diff, difflen)
    u = u.reshape(m, 1)
    ut = np.transpose(u)
    one = np.eye(m, dtype = float)
    H = one - np.dot(2,(np.dot(u, ut)))
    #print(H)
    Hx = np.dot(H, vec)
    #print(Hx)
    return Hx, H #could've returned y, it's equal to Hx
    
def length(vec):
    xlen = 0 
    for xi in vec:
        xlen += (xi**2)
    xlen = np.sqrt(xlen)
    return xlen


In [3]:
vector = np.zeros((5,), dtype = float)
vector[:] = 1
print(length(vector))
#print(vector)
y, H = householder(vector)
print(y)


2.23606797749979
[[2.23606798e+00]
 [3.33066907e-16]
 [2.77555756e-16]
 [2.77555756e-16]
 [3.33066907e-16]]


Test your function using tests below:

In [4]:
# Test I.1 (10% of the total grade).
np.set_printoptions(suppress=True)
v = np.array([1, 2, 3])
v1, h = householder(v)
v = v.reshape((v.shape[0], 1))

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [5]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)
vec = vec.reshape((vec.shape[0], 1))
assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [117]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    dim = max(n, m)
    Q = np.eye(dim)
    for i in range(n):
        col = a1[i:, i]
        Hcol, H = householder(col)
        matr = np.eye(dim) #fill empty spaces with I matrices
        matr[i:, i:] = H[:,:] #expand dims since each next H matrix is smaller then the previous one
        a1 = matr @ a1 
        Q = Q @ matr
    return Q, a1

In [118]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [119]:
#test
rndm = np.random.RandomState(4321)
a = rndm.uniform(size=(5, 5))
q, r = qr_decomp(a)

print("Q: \n{0} \n".format(q))
print("R: \n{0} \n".format(r))
print("Q*R:\n{0} \n".format(np.dot(q,r)))
print("A: \n{0} \n".format(a))
print("Q*R and A are identical")

Q: 
[[ 0.03996877  0.83611814  0.33730082  0.36831611 -0.22333909]
 [ 0.55260347 -0.19638016  0.35607705 -0.33260121 -0.64703153]
 [ 0.34938941  0.09561146 -0.81959218  0.29385966 -0.33271708]
 [ 0.53646211 -0.31305083  0.26763676  0.63436251  0.37438263]
 [ 0.53213471  0.39395293 -0.12679321 -0.51473331  0.52972337]] 

R: 
[[ 1.77145504  1.06347587  1.36290265  0.69797012  0.81301097]
 [ 0.          0.92398209  0.69369165  0.31329159  0.17499983]
 [ 0.          0.          0.39556159 -0.36050868 -0.31701935]
 [ 0.         -0.         -0.          0.3206718   0.24527222]
 [-0.         -0.          0.          0.         -0.13822886]] 

Q*R:
[[0.07080288 0.81506401 0.76790496 0.2863545  0.19309431]
 [0.97891221 0.40622871 0.75776787 0.08915177 0.30988347]
 [0.61892762 0.45991048 0.21830943 0.66352023 0.6786827 ]
 [0.95031851 0.28126114 0.6198517  0.38329512 0.40036072]
 [0.94265271 0.92991787 0.94837513 0.37548586 0.34229612]] 

A: 
[[0.07080288 0.81506401 0.76790496 0.2863545  0.193094

In [9]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [10]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
print("scipy: \n")
print("Q: \n{0} \n".format(qq))
print("R: \n{0} \n".format(rr))
q, r = qr_decomp(a)
print("me: \n")
print("Q: \n{0} \n".format(q))
print()
print("R: \n{0} \n".format(r))

scipy: 

Q: 
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

R: 
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]] 

me: 

Q: 
[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 


R: 
[[ 1.40152769  1.2514379   0.89523615]
 [-0.          0.84158252  0.68447942]
 [ 0.          0.          0.5496046 ]
 [ 0.         -0.

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

***My Q and R don't agree with scipy's QQ and RR because QR-decomposition of a matrix is not unique in that particular case and scipy probably uses some other version of an algorithm***

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [114]:
def householder_u(vec): #returns vector u instead of matrix H 
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype = float)
    m = vec.shape[0]
    if m == 1:
        e = np.asarray([1])
        e = e.reshape((e.shape[0], 1))
        return np.asarray(vec), e
    #print(vec)
    vec = vec.reshape((m, 1)) #had to expand dims to use np.transpose
    xlen = length(vec)
    y = np.zeros(vec.shape, dtype = float)
    y[0] = xlen
    
    numerator = 0 #avoiding the roundoff error
    for xi in vec[1:]:
        numerator -= (xi**2)
    denominator = vec[0] + xlen
    diff_x1_y = numerator/denominator
    diff_rest = vec[1:]
    diff = np.insert(diff_rest, 0, diff_x1_y)
    difflen = length(diff)
    
    u = np.divide(diff, difflen)
    u = u.reshape(m, 1)
    
    return y, u

In [128]:
def qr_decomp_2(A): # I understand how to get R but I don't see how can we get Q from reflection vectors
    A1 = np.array(A, copy=True, dtype=float)
    m, n = np.shape(A1)
    dim = max(m, n)
    one = np.eye(dim, dtype = float)
    u_arr = []
    for i in range(n): 
        col = A1[i:, i]
        ncol, u = householder_u(col) #get a reflection vector for each column of A
        ncol = np.reshape(ncol, (ncol.shape[0], )) 
        A1[i:, i] = ncol #replace i-th column with new column 
        u_arr.append(u)
        
    return A1, u_arr

def mul_Qt(A, u_arr):
    pass
    
def rest(u, col):
    m = u.shape[0]
    one = np.ones((m,1))
    utx = np.transpose(u) @ col
    duutx = 2*u @ utx
    return one - duutx

        
   

In [138]:
n = 5
rndm = np.random.RandomState(4321)
A = rndm.uniform(size=(n, n))
R, u_arr = qr_decomp_2(A)
print("A matrix: \n{0}\n".format(A))
print("R matrix: \n{0}\n".format(R))
print("Array of reflection vectors:")
for u in u_arr:
    print("{0} \n".format(u))


A matrix: 
[[0.07080288 0.81506401 0.76790496 0.2863545  0.19309431]
 [0.97891221 0.40622871 0.75776787 0.08915177 0.30988347]
 [0.61892762 0.45991048 0.21830943 0.66352023 0.6786827 ]
 [0.95031851 0.28126114 0.6198517  0.38329512 0.40036072]
 [0.94265271 0.92991787 0.94837513 0.37548586 0.34229612]]

R matrix: 
[[1.77145504 0.81506401 0.76790496 0.2863545  0.19309431]
 [0.         1.1490842  0.75776787 0.08915177 0.30988347]
 [0.         0.         1.15381564 0.66352023 0.6786827 ]
 [0.         0.         0.         0.5365676  0.40036072]
 [0.         0.         0.         0.         0.34229612]]

Array of reflection vectors:
[[-0.69283159]
 [ 0.39880072]
 [ 0.25214598]
 [ 0.38715188]
 [ 0.3840289 ]] 

[[-0.56854029]
 [ 0.35198991]
 [ 0.21526164]
 [ 0.71170744]] 

[[-0.63670774]
 [ 0.42187253]
 [ 0.64546636]] 

[[-0.37792434]
 [ 0.92583648]] 

[[1]] 



In [146]:
def cheto(A, u_arr): 
    A1 = np.array(A, copy=True, dtype=float)
    m, n = np.shape(A1)
    dim = max(m, n)
    Q = np.eye(m, dtype = float)
    for i in range(n): 
        col = A1[i:, i]
        col = np.reshape(col, (col.shape[0], 1))
        newcol = rest(u_arr[i], col)
        newcol = np.reshape(newcol, (col.shape[0], ))
        Q[i:, i] = newcol
    return Q

array([[4.78408389, 2.20120439, 2.0738442 , 0.77334391, 0.52148057],
       [0.03735607, 2.0198756 , 1.3368733 , 0.16141723, 0.54415403],
       [0.67505258, 0.93120672, 2.93510719, 1.44151925, 1.5545423 ],
       [0.08800853, 0.86638506, 1.02140893, 0.94934808, 0.95204357],
       [0.10158815, 0.12727198, 0.15670883, 0.39201779, 0.42582142]])

In [147]:
Q = cheto(A, u_arr)
possibly_R = np.transpose(Q) @ A
print("Qt @ A and R must be identical \n")
print("Qt @ A: \n", possibly_R)
print("R: ", R)

Qt @ A and R must be identical 

Qt @ A: 
 [[0.54898508 2.45233177 2.25819744 1.06864892 0.82616283]
 [2.78947278 1.22371471 1.95056217 0.81554291 1.21837516]
 [1.60786455 1.04508742 0.70713456 1.44934093 1.48346201]
 [1.68467612 0.90511759 1.30713158 0.67653994 0.67549376]
 [0.29731999 0.29330332 0.29912488 0.11843116 0.10796285]]
R:  [[1.77145504 0.81506401 0.76790496 0.2863545  0.19309431]
 [0.         1.1490842  0.75776787 0.08915177 0.30988347]
 [0.         0.         1.15381564 0.66352023 0.6786827 ]
 [0.         0.         0.         0.5365676  0.40036072]
 [0.         0.         0.         0.         0.34229612]]


***AT LEAST I TRIED***